# Toxic Comment Classification Challenge: Starter Colab Notebook

leduykhanh@

Short Link: **tiny.cc/mlcc-toxic-starter-colab**

Kaggle challenge: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge


>[Toxic Comment Classification Challenge: Starter Colab Notebook](#scrollTo=SXgAs6uoQ_Zv)

>>[Import the dataset](#scrollTo=pdQHcSnWdM4b)

>>[Input and Label](#scrollTo=rkGUNgjocwKi)

>>[Train/Validation Split](#scrollTo=qaUp1VaFek9e)

>>[Train a model](#scrollTo=7C1075_ce57W)

>>[References](#scrollTo=7C1075_ce57W)



### Team members:
- Deepika, YeeJie, Kok Meng

## Import the dataset

In [0]:
# Original data belongs to https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data
! wget https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv

--2018-11-23 08:07:01--  https://www.dropbox.com/s/ggl9krhh6dcwhhz/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ggl9krhh6dcwhhz/train.csv [following]
--2018-11-23 08:07:01--  https://www.dropbox.com/s/raw/ggl9krhh6dcwhhz/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3b37888dfb3d5d64949572009e.dl.dropboxusercontent.com/cd/0/inline/AWLIjOa4bpm-Lo7boBoe8gZWEc-OrslTcsZmYRVaxqyp8_N3YtMkOJrIBiLEbn-xBUhSpNJY8T2xWprbf5dvQf-HVYhB5NEUPAt59IYyoz3I7kAiuKrpRQj5yK-8TBQQVpXdRUT5LTOipv_UAK7Kde3rpvxBwHBPL8ezaJa1CgrKzWRoN9gukIducaqyiL3tGaVxUb2QA3dhrHJXYgnqmZx_/file [following]
--2018-11-23 08:07:01--  https://uc3b37888dfb3d5d64949572009e.dl.dropboxusercontent.com/cd/0/inline/AWLIjOa4bpm-Lo7boBoe8gZWEc-OrslTcsZmYRV

In [0]:
! ls -ls train.csv

67196 -rw-r--r-- 1 root root 68802655 Nov 23 08:07 train.csv


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
%mkdir data/

In [0]:
#2. Get the file
downloaded = drive.CreateFile({'id': <file_id>})
downloaded.GetContentFile('data/glove.6B.100d.txt')

## Start here

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print('TensorFlow Version: {}'.format(tf.__version__))

Found GPU at: /device:GPU:0
TensorFlow Version: 1.12.0


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_auc_score

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Embedding, Dense, Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout
from keras.layers import Bidirectional, LSTM, GRU, TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import SpatialDropout1D

Using TensorFlow backend.


## Input and Label

Input: "comment_text"

Label: "toxic"

## Train/Validation Split

In [0]:
data = pd.read_csv("train.csv")

# Train / validation split
dataset = data[ ["comment_text", "toxic"] ]
dataset = dataset.sample(frac=1, random_state=0)
idx = int(0.8 * len(dataset))

train_dataset = dataset.head(int(0.8 * len(dataset)))
validation_dataset = dataset.tail(int(0.2 * len(dataset)))

In [0]:
print(len(dataset))
print(len(train_dataset))
print(len(validation_dataset))

159571
127656
31914


In [0]:
train_dataset.describe()

,toxic
count,127656.000000
mean,0.095836
std,0.294367
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [0]:
validation_dataset.describe()

,toxic
count,31914.000000
mean,0.095883
std,0.294435
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [0]:
train_dataset.head(10)

,comment_text,toxic
74251,"""\nI haven't paraphrased you at all, Gary. Yo...",0
131406,I BLOCKED REVERS! I BLOCKED REVERS! I BLOCKED ...,1
120969,I'm sorry. I'd like to unreservedly retract my...,0
121827,I don't know if this is exactly like the Press...,0
4771,"Thank you all, we'll all improve the Wikipedia...",0
79050,"I removed\nIf V is separable, it follows that ...",0
86098,Leave your emotions out of it. That is the exa...,0
55703,another thing this article needs is a descript...,0
113939,"""\nYou may take my word for it that it is. and...",0
19481,"20, 19 March 2010 (UTC)\nI do. This isn't a sc...",0


In [0]:
validation_dataset.head(10)

,comment_text,toxic
7612,Melt / flow \n\nRe . I think it is flow; you c...,0
72588,"""FAOL|Arabic|ar:لون}}\n\nPLEASE NOTE that the ...",0
156251,""" (UTC)\n\nI have a friend who got 3 years of ...",0
18619,Re: Banksia photos \n\nThanks for the hint. I'...,0
133070,Hi Gaff - thanks for the reply. I certainly ag...,0
140026,"""\n\n I'm strongly opposed to any furtherance ...",0
84061,"Thank you, . That sounds like what I was think...",0
88905,"""\nYou may be interested in Wikipedia:Example ...",0
142308,Was that necessary? \n\nDo not mess with me or...,1
131929,"Nice. \n\nNikosilver, you have made a nice rev...",0


## Train a model

Your model's code start here. All the best :)



## References

* Text classification using tf.keras https://www.tensorflow.org/tutorials/keras/basic_text_classification

* How to build a simple text classifier with TF-Hub https://www.tensorflow.org/tutorials/text_classification_with_tf_hub

* Text Classification Tutorial using Tensorflow: https://mohammadkhalifa.github.io/Step-by-Step-Text-Classification-Tutorial-using-Tensorflow/
  * Feature-engineering methods for text classification https://towardsdatascience.com/understanding-feature-engineering-part-3-traditional-methods-for-text-data-f6f7d70acd41


* Tensorflow Estimator API: https://www.tensorflow.org/tutorials/recurrent

* Using pre-trained word embeddings in a Keras model https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

* Text Classification with TensorFlow Estimators http://ruder.io/text-classification-tensorflow-estimators/

* Scikit-Learns NLP documentation: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

* [Review on  Kaggle Toxic Comment Classification Challenge](https://medium.com/the-artificial-impostor/review-kaggle-toxic-comment-classification-challenge-part-1-934447339309)


In [0]:
import re

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = re.sub(r"\n", " ", string)
    string = re.sub(r"[^\w\s]", "", string)
    return string.strip().lower()


texts = []
for i in range(dataset.shape[0]):
    texts.append(clean_str(dataset['comment_text'].iloc[i]))

labels = to_categorical(dataset['toxic'].values)
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (159571, 2)


In [0]:
MAX_SEQUENCE_LENGTH = 250
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)

In [0]:
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)


# Splitting into train and validation sets
x_train = data[:idx]
y_train = labels[:idx]
x_val = data[idx:]
y_val = labels[idx:]

print('Number of toxic comments in training and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Found 253536 unique tokens.
Shape of data tensor: (159571, 250)
Number of toxic comments in training and validation set 
[115422.  12234.]
[28855.  3060.]


In [0]:
# totalNumWords = [len(x) for x in sequences]
# plt.hist(totalNumWords, bins=np.arange(0,410,10))
# plt.show()

In [0]:
embeddings_index = {}
with open('data/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [0]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

## Model 1

In [0]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = Embedding(MAX_NUM_WORDS, EMBEDDING_DIM)(sequence_input)

kernel_sizes = [3,4,5]
convs = []
for fsz in kernel_sizes:
    l_conv = Conv1D(filters=54, kernel_size=fsz, activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(pool_size=5)(l_conv)
    convs.append(l_pool)
    
l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(filters=64, kernel_size=5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(pool_size=5)(l_cov1)
l_cov2 = Conv1D(filters=64, kernel_size=5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(pool_size=5)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(units=64, activation='relu')(l_flat)
preds = Dense(units=2, activation='sigmoid')(l_dense)

model1 = Model(sequence_input, preds)
model1.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0003),
              metrics=['accuracy'])
print(model1.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 250, 100)     2000000     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 248, 54)      16254       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 247, 54)      21654       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [0]:
epochs = 10
batch_size = 1024

file_path = 'weights1.h5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=3)

callbacks_list = [checkpoint, early]
model1.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           shuffle=True,
           validation_data=(x_val, y_val),
           callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 27s 210us/step - loss: 0.3547 - acc: 0.8958 - val_loss: 0.2852 - val_acc: 0.9041

Epoch 00001: val_loss improved from inf to 0.28522, saving model to weights1.h5
Epoch 2/10
127656/127656 [==============================] - 23s 178us/step - loss: 0.2503 - acc: 0.9048 - val_loss: 0.2144 - val_acc: 0.9123

Epoch 00002: val_loss improved from 0.28522 to 0.21443, saving model to weights1.h5
Epoch 3/10
127656/127656 [==============================] - 23s 178us/step - loss: 0.1551 - acc: 0.9414 - val_loss: 0.1483 - val_acc: 0.9473

Epoch 00003: val_loss improved from 0.21443 to 0.14830, saving model to weights1.h5
Epoch 4/10
127656/127656 [==============================] - 23s 177us/step - loss: 0.1122 - acc: 0.9582 - val_loss: 0.1471 - val_acc: 0.9491

Epoch 00004: val_loss improved from 0.14830 to 0.14706, saving model to weights1.h5
Epoch 5/10
127656/127656 [=======================

In [0]:
y_hat1 = model1.predict(x_val)
roc_auc_score(y_val, y_hat1)

0.9274868567539072

## Model 2

In [0]:
# define model architecture
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = Embedding(MAX_NUM_WORDS, EMBEDDING_DIM)(sequence_input)
x = Conv1D(64, 3, activation='relu')(embedded_sequences)
x = MaxPooling1D(4)(x)
x = GRU(64, dropout=0.1, recurrent_dropout=0.5)(x) # defaults include tanh activation
preds = Dense(2, activation='sigmoid')(x)

model2 = Model(sequence_input, preds)
model2.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.0003),
              metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 250)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 250, 100)          2000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 248, 64)           19264     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 62, 64)            0         
_________________________________________________________________
gru_3 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 2,044,162
Trainable params: 2,044,162
Non-trainable params: 0
_________________________________________________________________


In [0]:
epochs = 10
batch_size = 1024

file_path = 'weights2.h5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=3)

callbacks_list = [checkpoint, early]
model2.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           shuffle=True,
           validation_data=(x_val, y_val),
           callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 31s 244us/step - loss: 0.3915 - acc: 0.8980 - val_loss: 0.2671 - val_acc: 0.9041

Epoch 00001: val_loss improved from inf to 0.26707, saving model to weights2.h5
Epoch 2/10
127656/127656 [==============================] - 30s 233us/step - loss: 0.2162 - acc: 0.9182 - val_loss: 0.1573 - val_acc: 0.9436

Epoch 00002: val_loss improved from 0.26707 to 0.15726, saving model to weights2.h5
Epoch 3/10
127656/127656 [==============================] - 29s 230us/step - loss: 0.1246 - acc: 0.9555 - val_loss: 0.1222 - val_acc: 0.9570

Epoch 00003: val_loss improved from 0.15726 to 0.12216, saving model to weights2.h5
Epoch 4/10
127656/127656 [==============================] - 30s 232us/step - loss: 0.1030 - acc: 0.9636 - val_loss: 0.1210 - val_acc: 0.9583

Epoch 00004: val_loss improved from 0.12216 to 0.12097, saving model to weights2.h5
Epoch 5/10
127656/127656 [=======================

In [0]:
y_hat2 = model2.predict(x_val)
print(roc_auc_score(y_val, y_hat2))

0.9625658804502566


## Model 3

In [0]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='sigmoid')(x)

model3 = Model(sequence_input, preds)
model3.compile(loss='binary_crossentropy',
               optimizer=Adam(lr=0.0005),
               metrics=['accuracy'])
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 250)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 250, 100)          25353700  
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 246, 128)          64128     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 49, 128)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 45, 128)           82048     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 9, 128)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 5, 128)            82048     
__________

In [0]:
epochs = 10
batch_size = 1024

file_path = 'weights3.h5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=3)

callbacks_list = [checkpoint, early]
model3.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           shuffle=True,
           validation_data=(x_val, y_val),
           callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 15s 118us/step - loss: 0.2658 - acc: 0.9100 - val_loss: 0.1909 - val_acc: 0.9297

Epoch 00001: val_loss improved from inf to 0.19093, saving model to weights3.h5
Epoch 2/10
127656/127656 [==============================] - 13s 103us/step - loss: 0.1646 - acc: 0.9393 - val_loss: 0.1491 - val_acc: 0.9454

Epoch 00002: val_loss improved from 0.19093 to 0.14907, saving model to weights3.h5
Epoch 3/10
127656/127656 [==============================] - 13s 103us/step - loss: 0.1432 - acc: 0.9474 - val_loss: 0.1417 - val_acc: 0.9484

Epoch 00003: val_loss improved from 0.14907 to 0.14170, saving model to weights3.h5
Epoch 4/10
127656/127656 [==============================] - 13s 103us/step - loss: 0.1321 - acc: 0.9509 - val_loss: 0.1399 - val_acc: 0.9492

Epoch 00004: val_loss improved from 0.14170 to 0.13988, saving model to weights3.h5
Epoch 5/10
127656/127656 [=======================

In [0]:
y_hat3 = model3.predict(x_val)
print(roc_auc_score(y_val, y_hat3))

0.9537060131624995


## Model 4

In [0]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(embedded_sequences)
x = Bidirectional(GRU(80, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = Concatenate(axis=1)([avg_pool, max_pool])
preds = Dense(2, activation="sigmoid")(conc)

model4 = Model(sequence_input, preds)
model4.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])
print(model4.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 100)     25353700    input_8[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 250, 100)     0           embedding_1[5][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 250, 160)     86880       spatial_dropout1d_3[0][0]        
__________________________________________________________________________________________________
global_ave

In [0]:
epochs = 10
batch_size = 1024

file_path = 'weights4.h5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                             save_best_only=True, mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=3)

callbacks_list = [checkpoint, early]
model4.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           shuffle=True,
           validation_data=(x_val, y_val),
           callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 146s 1ms/step - loss: 0.2745 - acc: 0.9041 - val_loss: 0.1673 - val_acc: 0.9406

Epoch 00001: val_loss improved from inf to 0.16727, saving model to weights4.h5
Epoch 2/10
127656/127656 [==============================] - 143s 1ms/step - loss: 0.1488 - acc: 0.9455 - val_loss: 0.1287 - val_acc: 0.9523

Epoch 00002: val_loss improved from 0.16727 to 0.12873, saving model to weights4.h5
Epoch 3/10
127656/127656 [==============================] - 144s 1ms/step - loss: 0.1341 - acc: 0.9507 - val_loss: 0.1235 - val_acc: 0.9541

Epoch 00003: val_loss improved from 0.12873 to 0.12350, saving model to weights4.h5
Epoch 4/10
127656/127656 [==============================] - 144s 1ms/step - loss: 0.1298 - acc: 0.9519 - val_loss: 0.1177 - val_acc: 0.9560

Epoch 00004: val_loss improved from 0.12350 to 0.11766, saving model to weights4.h5
Epoch 5/10
127656/127656 [===========================

In [0]:
# model4.load_weights('weights4.h5')

In [0]:
y_hat4 = model4.predict(x_val)
roc_auc_score(y_val, y_hat4)

0.9747086486070198

## Ensemble = model 2 + 4

In [0]:
y_overall = (y_hat2 + y_hat4) / 2
print(roc_auc_score(y_val, y_overall))

0.9767443228085435


# Test Set evaluation
Your boss is now ready to look at your model. However, they want to evaluate your model on a separate test set before deploying it. There's one small catch. It's possible that you've tuned your hyper-parameters a lot to lower the error on the validation set. Let's see if this affects you :-) .

In [0]:
# Original data belongs to https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data
! wget https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv

--2018-11-23 08:44:48--  https://www.dropbox.com/s/tst2y6mzwzbhxo3/test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tst2y6mzwzbhxo3/test.csv [following]
--2018-11-23 08:44:49--  https://www.dropbox.com/s/raw/tst2y6mzwzbhxo3/test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaead5835e07aa74609cb330f4f.dl.dropboxusercontent.com/cd/0/inline/AWJqVC91HRWyZJUmp-4TLnKiH9eOAX8Hrv8ODsVW0pmHnG1lE1kiNoDG8No8kuGMQkrghY-sqPRDw63cH0rvuMUJcdTQmq4n1w_OXO8PU0XfUP2MtsxVsFJt4xylDskoa6FwvfwMnIdtms7h4cs1QK5RVQVNoh7po2fzty9LKPuaZWPD-dLXsa94O5_cJmj8CiERVAgoDzOgQbeHj1FymNEE/file [following]
--2018-11-23 08:44:49--  https://ucaead5835e07aa74609cb330f4f.dl.dropboxusercontent.com/cd/0/inline/AWJqVC91HRWyZJUmp-4TLnKiH9eOAX8Hrv8ODsVW0p

In [0]:
import pandas as pd
test_dataset = pd.read_csv("test.csv")[ ["comment_text", "toxic"] ]
test_dataset.describe()

,toxic
count,63978.000000
mean,0.095189
std,0.293478
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [0]:
y_test = to_categorical(test_dataset['toxic'].values)
print('Shape of label tensor:', y_test.shape)

test_texts = []
for i in range(test_dataset.shape[0]):
    test_texts.append(clean_str(test_dataset['comment_text'].iloc[i]))

test_sequences = tokenizer.texts_to_sequences(test_texts)

x_test = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_test.shape)

Shape of label tensor: (63978, 2)
Shape of data tensor: (63978, 250)


In [0]:
# Model 2
y_hat_test2 = model2.predict(x_test)
roc_auc_score(y_test, y_hat_test2)

0.9516816644859084

In [0]:
# Model 4
y_hat_test4 = model4.predict(x_test)
roc_auc_score(y_test, y_hat_test4)

0.9581478008096264

In [0]:
# Ensemble
y_overall = (y_hat_test2 + y_hat_test4) / 2
print("Final AUC on test set: {:.5f}".format(roc_auc_score(y_test, y_overall)))

Final AUC on test set: 0.96301
